In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/'My Drive'

Mounted at /content/gdrive
/content/gdrive/My Drive


1. Import correctly labeled data and corresponding summary data.

In [2]:
import pandas as pd
from os import path
import os, zipfile

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [8]:
#Load bots with easy patterns in obvious_bots. 
obvious_bots = pd.read_csv('https://raw.githubusercontent.com/griffinbaum22/bot_detection/main/obvious_bots').obvious_bots.values
user_df = pd.read_csv('https://raw.githubusercontent.com/griffinbaum22/bot_detection/main/user_df').drop(columns= 'Unnamed: 0')
user_df.describe()

,total_trxs,total_wins,win_ratio,total_bet,total_recieved,net_profit,days_active,unique_hrs_active,avg_hrs_active,avg_bet_size
count,1657.000000,1657.000000,1657.000000,1657.000000,1657.000000,1657.000000,1657.000000,1657.000000,1657.000000,1657.000000
mean,459.931201,402.042849,0.855654,664.813015,670.873814,6.060799,19.680145,2.792396,1.313612,5.114451
std,8061.353947,7637.514270,0.226804,6579.566037,6286.401623,695.669483,13.950319,3.368168,1.197422,69.715415
min,1.000000,0.000000,0.000000,0.000100,0.000000,-17244.700900,1.000000,1.000000,1.000000,0.000100
25%,16.000000,11.000000,0.875000,3.200000,2.989900,-0.210100,2.000000,2.000000,1.000000,0.100000
50%,32.000000,30.000000,0.937500,3.200000,3.196100,-0.003900,31.000000,2.000000,1.032258,0.100000
75%,32.000000,31.000000,0.968750,3.200000,3.299200,0.099200,31.000000,2.000000,1.032258,0.247200
max,254893.000000,242135.000000,1.000000,186814.971600,177763.788500,11860.576600,31.000000,24.000000,23.967742,2458.611313


In [10]:
# Load research data with reliable labels in labels.
column_names = ['user', 'detection']
reliable_labels = pd.read_csv('reliable-labels.csv', names=column_names)
reliable_labels

,user,detection
0,gu3dimzvguge,bot
1,g42tgmbshege,bot
2,jixiango3133,bot
3,guydknzsgege,bot
4,gyztqnjwg4ge,bot
...,...,...
1178,afd4ctrpcer1,bot
1179,beirwpuugtkm,bot
1180,ycdzhtpheges,bot
1181,4lp3gnxrbfyc,bot


In [11]:
# Combine realiable_labels with obvious_bots in combined1. 
combined1 = user_df.loc[user_df.name.isin(reliable_labels.user.values)]
combined1['detection'] = combined1.name.apply(lambda user: reliable_labels.loc[reliable_labels.user == user, 'detection'].values[0])
combined1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,name,total_trxs,total_wins,win_ratio,total_bet,total_recieved,net_profit,days_active,unique_hrs_active,avg_hrs_active,avg_bet_size,detection
7,zhaowenyoung,1,1.0,1.000000,0.1000,0.1031,0.0031,1,1,1.000000,0.100000,bot
8,sourceeoseos,1,1.0,1.000000,0.1000,0.1031,0.0031,1,1,1.000000,0.100000,bot
11,hchn12345432,1,1.0,1.000000,0.1000,0.1884,0.0884,1,1,1.000000,0.100000,bot
12,shadowalker3,1,1.0,1.000000,0.1000,0.1101,0.0101,1,1,1.000000,0.100000,bot
14,zjgzjhshy123,1,1.0,1.000000,0.1000,4425.8081,4425.7081,1,1,1.000000,0.100000,bot
...,...,...,...,...,...,...,...,...,...,...,...,...
1634,gy3tamrxgqge,3838,3637.0,0.947629,1849.4000,1813.2541,-36.1459,3,10,3.333333,0.360544,bot
1640,g44dmnrxg4ge,5791,4236.0,0.731480,108699.5978,99322.5100,-9377.0878,26,19,4.423077,26.704593,bot
1641,g44temrwgage,5922,3421.0,0.577676,41020.3531,41024.0871,3.7340,16,21,4.187500,3.576469,bot
1644,stardlullaby,7464,6611.0,0.885718,6016.6742,6120.0951,103.4209,10,17,4.800000,1.143940,bot


### 2. Build models on research data. 

In [12]:
combined1.detection.value_counts()

bot      1167
human       2
Name: detection, dtype: int64

In [13]:
threshold = len(combined1[combined1['detection']=='bot'])/len(combined1)
threshold

0.998289136013687

In [14]:
selected_columns = ['total_trxs', 'win_ratio', 'days_active','unique_hrs_active', 'avg_hrs_active',  'avg_bet_size']
X = combined1[selected_columns]
y = combined1['detection']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_test, y_test, test_size=0.25, random_state=42)


In [15]:
logreg = LogisticRegression()                              
logreg.fit(X_train, y_train)                                 
y_prediction = logreg.predict(X_validation)                             
acc_logreg = sum(y_prediction == y_validation)/len(y_validation)*100    

print('Logistic Regression validation accuracy:', str(round(acc_logreg,2)),'%')

Logistic Regression validation accuracy: 100.0 %


In [16]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_prediction = neigh.predict(X_test) 
acc_neigh = sum(y_prediction == y_test)/len(y_test)*100

print('KNN validation accuracy:', str(round(acc_neigh,2)),'%')

KNN validation accuracy: 100.0 %


In [17]:
percept = Perceptron()
percept.fit(X_train, y_train)
y_prediction = percept.predict(X_validation) 
acc_percept = sum(y_prediction == y_validation)/len(y_validation)*100

print('Perceptron validation accuracy:', str(round(acc_percept,2)),'%')

Perceptron validation accuracy: 100.0 %


In [18]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_prediction = rf.predict(X_validation) 
acc_rf = sum(y_prediction == y_validation)/len(y_validation)*100

print('Random Forest validation accuracy:', str(round(acc_rf,2)),'%')

Random Forest validation accuracy: 100.0 %


### 3. Import Manually labeled data. 

In [19]:
# Load manually labeled data in manual
manual = pd.read_csv('team_labels.csv', names=column_names)
manual

,user,detection
0,12345dancing,human
1,12345haoming,human
2,14wkee4teosd,human
3,1af4gv2nrpw5,bot
4,1aisuion2gdh,bot
...,...,...
557,zkt55e1hnzty,bot
558,zm5caotinggg,human
559,zn12xtkgrcau,bot
560,zpejevem4pst,bot


In [ ]:
# Clean and extract all features.
def clean_file(file):
    # the current version of clean_file only checks for 
    banks = ['eossuperplay', 'endlessbank1', 'endlessdicex', 'endlesscrash']
    inspect = pd.read_json(file, lines=True)
    inspect['auth_actor'] = inspect['authorization'].apply(lambda x: x[0]['actor'])
    inspect['auth_perm'] = inspect['authorization'].apply(lambda x: x[0]['permission'])
    inspect = inspect.drop(columns=['authorization'])
    inspect = inspect[~inspect['d_to'].isin(banks)]
    return inspect

def summary(data):
    for col in data.columns:
        print(col)
        display(data[col].value_counts())
        print('\n')

archive = zipfile.ZipFile('endlessdice-201903.zip', 'r')

# num_want is now many we want for now
num_files, num_want = len(archive.namelist()), 20
num_want = num_files

combined = clean_file(archive.open(archive.namelist()[0]))
for i, name in enumerate(archive.namelist()[1:]):
    j = i + 1
    if j % 10 == 1: 
        print("{} / {}".format(j, num_want + 1))
        print(name)
    if j > num_want:
        break
    temp = clean_file(archive.open(name))
    combined = combined.append(temp)
print("finished")

temporal = combined[['d_to', 'trx_timestamp']].rename(columns={'d_to': 'user', 'trx_timestamp': 'time'})
temporal['time'] = pd.to_datetime(temporal['time'])
temporal['weekday'] = temporal['time'].dt.weekday
temporal['hour'] = temporal['time'].dt.hour
temporal.head()

1 / 63
actions-endlessbank1-20190302.json


### 4. Feature Engineering

In [ ]:
# Combine manual labels with features. 
grouped = temporal.groupby('user').count()[['time']].reset_index()
featured = grouped.merge(manual, left_on = 'user', right_on = 'user')
featured

,user,time,detection
0,12345dancing,11,human
1,12345haoming,3,human
2,14wkee4teosd,7,human
3,1af4gv2nrpw5,32,bot
4,1aisuion2gdh,30,bot
...,...,...,...
557,zkt55e1hnzty,31,bot
558,zm5caotinggg,154,human
559,zn12xtkgrcau,30,bot
560,zpejevem4pst,32,bot


In [ ]:
#Save the number of days an account is active for transactions in a week in days_active
a = temporal.groupby(['user', 'weekday']).count().reset_index()
b = a.groupby('user').count().reset_index().merge(featured, left_on = 'user', right_on = 'user')
featured['days_active'] = b['weekday']
featured

,user,time,detection,days_active
0,12345dancing,11,human,3
1,12345haoming,3,human,1
2,14wkee4teosd,7,human,2
3,1af4gv2nrpw5,32,bot,7
4,1aisuion2gdh,30,bot,7
...,...,...,...,...
557,zkt55e1hnzty,31,bot,7
558,zm5caotinggg,154,human,3
559,zn12xtkgrcau,30,bot,7
560,zpejevem4pst,32,bot,7


In [ ]:
c = temporal.groupby(['user', 'hour']).count().reset_index()
d = c.groupby('user').count().reset_index().merge(featured, left_on = 'user', right_on = 'user')
featured['hours_active'] = c['weekday']
featured.describe()

,time,days_active,hours_active
count,562.000000,562.000000,562.000000
mean,694.715302,5.754448,24.112100
std,10474.908276,2.201164,48.412001
min,2.000000,1.000000,1.000000
25%,29.000000,5.250000,1.000000
50%,30.000000,7.000000,2.000000
75%,32.000000,7.000000,29.000000
max,242136.000000,7.000000,355.000000


### 5. Models built on the manually labeled data. 

In [ ]:
threshold = len(new[new['detection']=='bot'])/len(new)
threshold

0.8202846975088968

In [ ]:
selected_columns = ['time','days_active','hours_active']
X = new[selected_columns]
y = new['detection']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_test, y_test, test_size=0.25, random_state=42)

In [ ]:
logreg = LogisticRegression()                              
logreg.fit(X_train, y_train)                                 
y_prediction = logreg.predict(X_validation)                             
acc_logreg = sum(y_prediction == y_validation)/len(y_validation)*100    

print('Logistic Regression validation accuracy:', str(round(acc_logreg,2)),'%')

Logistic Regression validation accuracy: 97.14 %


In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_prediction = neigh.predict(X_test) 
acc_neigh = sum(y_prediction == y_test)/len(y_test)*100

print('KNN validation accuracy:', str(round(acc_neigh,2)),'%')

KNN validation accuracy: 88.89 %


In [ ]:
percept = Perceptron()
percept.fit(X_train, y_train)
y_prediction = percept.predict(X_validation) 
acc_percept = sum(y_prediction == y_validation)/len(y_validation)*100

print('Perceptron validation accuracy:', str(round(acc_percept,2)),'%')

Random Forest validation accuracy: 83.81 %


In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_prediction = rf.predict(X_validation) 
acc_rf = sum(y_prediction == y_validation)/len(y_validation)*100

print('Random Forest validation accuracy:', str(round(acc_rf,2)),'%')

# 6. Models built with both Research and Manual Labels

In [ ]:
team_labels = pd.read_csv('team_labels.csv', names=column_names)
all_labels = labels.append(team_labels.loc[~team_labels.user.isin(labels.user.values)])

full = user_df.loc[user_df.name.isin(all_labels.user.values)]
full['detection'] = full.name.apply(lambda user: all_labels.loc[all_labels.user == user, 'detection'].values[0])
full.detection.value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


bot      1213
human      75
Name: detection, dtype: int64

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, FunctionTransformer
from sklearn.compose import ColumnTransformer

selected_columns = ['total_trxs', 'total_wins', 'win_ratio', 'total_bet',
       'total_recieved', 'net_profit', 'days_active', 'unique_hrs_active',
       'avg_hrs_active', 'avg_bet_size']

def select_columns():
    ft = FunctionTransformer(lambda x: x[selected_columns])
    ft.get_feature_names = lambda: selected_columns
    return ft

def numerical_transforms():
    return ColumnTransformer([
        # Edit this section so that it includes your chosen features
        ('total_trxs', KBinsDiscretizer(n_bins=3, encode="ordinal",strategy="quantile"), ['total_trxs']),
        ('total_wins', KBinsDiscretizer(n_bins=3, encode="ordinal",strategy="quantile"), ['total_wins']),
        ('total_bet', StandardScaler(), ['total_bet']),
        ('total_recieved', StandardScaler(), ['total_recieved']),
        ('net_profit', StandardScaler(), ['net_profit']),
        ('days_active', KBinsDiscretizer(n_bins=4, encode="ordinal",strategy="quantile"), ['days_active']),
        ('unique_hrs_active', KBinsDiscretizer(n_bins=3, encode="ordinal",strategy="quantile"), ['unique_hrs_active']),
        ('avg_hrs_active', KBinsDiscretizer(n_bins=3, encode="ordinal",strategy="quantile"), ['avg_hrs_active']),
        ('avg_bet_size', StandardScaler(), ['avg_bet_size'])],
        remainder='passthrough')

In [ ]:
full_pipeline = make_pipeline(select_columns(), numerical_transforms())


full_pipeline.fit(full)
X = full_pipeline.transform(full)
y = full['detection']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_test, y_test, test_size=0.25, random_state=42)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.

In [ ]:
logreg = LogisticRegression()                              
logreg.fit(X_train, y_train)                                 
y_prediction = logreg.predict(X_validation)                             
acc_logreg = sum(y_prediction == y_validation)/len(y_validation)*100    

print('Logistic Regression validation accuracy:', str(round(acc_logreg,2)),'%')

Logistic Regression validation accuracy: 95.44 %


In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_prediction = neigh.predict(X_test) 
acc_neigh = sum(y_prediction == y_test)/len(y_test)*100

print('KNN validation accuracy:', str(round(acc_neigh,2)),'%')

KNN validation accuracy: 95.06 %


In [ ]:
percept = Perceptron()
percept.fit(X_train, y_train)
y_prediction = percept.predict(X_validation) 
acc_percept = sum(y_prediction == y_validation)/len(y_validation)*100

print('Perceptron validation accuracy:', str(round(acc_percept,2)),'%')

Perceptron validation accuracy: 95.02 %


In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_prediction = rf.predict(X_validation) 
acc_rf = sum(y_prediction == y_validation)/len(y_validation)*100

print('Random Forest validation accuracy:', str(round(acc_rf,2)),'%')

Random Forest validation accuracy: 94.61 %
